# 🤖 RL Football Champions - GPU Training

This notebook trains the DQN agent on Google Colab's GPU for much faster training.

**Instructions:**
1. Go to Runtime → Change runtime type → Select **GPU**
2. Run all cells (Runtime → Run all)
3. Wait for training to complete
4. Download the weights file

Expected time: ~2-4 hours for 100k episodes

In [ ]:
# Check GPU availability
import tensorflow as tf
print(f"TensorFlow version: {tf.__version__}")
print(f"GPU available: {tf.config.list_physical_devices('GPU')}")
if tf.config.list_physical_devices('GPU'):
    print("✅ GPU is ready!")
else:
    print("⚠️ No GPU detected. Go to Runtime → Change runtime type → GPU")

In [ ]:
import numpy as np
import json
import time
from collections import deque
import random
import keras
from keras import layers, ops

print("✅ Libraries imported")

## Game Simulation

In [ ]:
class Player:
    def __init__(self, x, y, team):
        self.x = x
        self.y = y
        self.start_x = x
        self.start_y = y
        self.team = team
        self.vx = 0.0
        self.vy = 0.0
        self.radius = 25
        self.speed = 4
        self.friction = 0.85
        self.kick_animation = 0

    def reset(self):
        self.x = self.start_x
        self.y = self.start_y
        self.vx = 0.0
        self.vy = 0.0

    def update(self):
        self.x += self.vx
        self.y += self.vy
        self.vx *= self.friction
        self.vy *= self.friction

    def move(self, dx, dy):
        self.vx += dx * self.speed * 0.5
        self.vy += dy * self.speed * 0.5
        current_speed = np.sqrt(self.vx**2 + self.vy**2)
        if current_speed > self.speed:
            self.vx = (self.vx / current_speed) * self.speed
            self.vy = (self.vy / current_speed) * self.speed

    def kick(self):
        self.kick_animation = 1


class Ball:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        self.start_x = x
        self.start_y = y
        self.vx = 0.0
        self.vy = 0.0
        self.radius = 12

    def reset(self):
        self.x = self.start_x
        self.y = self.start_y
        self.vx = 0.0
        self.vy = 0.0


class Game:
    def __init__(self, match_time=30):
        self.width = 800
        self.height = 500
        self.padding = 40

        self.field_left = self.padding
        self.field_right = self.width - self.padding
        self.field_top = self.padding
        self.field_bottom = self.height - self.padding
        self.field_width = self.field_right - self.field_left
        self.field_height = self.field_bottom - self.field_top

        self.goal_height = 120
        self.goal_y = self.height / 2

        self.blip = Player(self.field_left + 80, self.height / 2, 'blip')
        self.bloop = Player(self.field_right - 80, self.height / 2, 'bloop')
        self.ball = Ball(self.width / 2, self.height / 2)

        self.blip_score = 0
        self.bloop_score = 0
        self.match_time = match_time
        self.time_remaining = match_time

    def reset(self):
        self.blip.reset()
        self.bloop.reset()
        self.ball.reset()
        self.time_remaining = self.match_time

    def reset_scores(self):
        self.blip_score = 0
        self.bloop_score = 0

    def update(self):
        self.time_remaining -= 1/60

        self.blip.update()
        self.bloop.update()
        self._update_ball()
        self._check_collision(self.blip)
        self._check_collision(self.bloop)
        self._constrain_player(self.blip)
        self._constrain_player(self.bloop)

        event = self._check_goal()
        done = self.time_remaining <= 0

        return event, done

    def _update_ball(self):
        self.ball.x += self.ball.vx
        self.ball.y += self.ball.vy
        self.ball.vx *= 0.98
        self.ball.vy *= 0.98

        if self.ball.y - self.ball.radius < self.field_top:
            self.ball.y = self.field_top + self.ball.radius
            self.ball.vy *= -0.8
        if self.ball.y + self.ball.radius > self.field_bottom:
            self.ball.y = self.field_bottom - self.ball.radius
            self.ball.vy *= -0.8

        in_goal_range = (self.goal_y - self.goal_height/2 < self.ball.y < self.goal_y + self.goal_height/2)
        if not in_goal_range:
            if self.ball.x - self.ball.radius < self.field_left:
                self.ball.x = self.field_left + self.ball.radius
                self.ball.vx *= -0.8
            if self.ball.x + self.ball.radius > self.field_right:
                self.ball.x = self.field_right - self.ball.radius
                self.ball.vx *= -0.8

    def _check_collision(self, player):
        dx = self.ball.x - player.x
        dy = self.ball.y - player.y
        dist = np.sqrt(dx**2 + dy**2)
        min_dist = player.radius + self.ball.radius

        if dist < min_dist and dist > 0:
            nx = dx / dist
            ny = dy / dist
            self.ball.x = player.x + nx * min_dist
            self.ball.y = player.y + ny * min_dist
            kick_power = 12 if player.kick_animation > 0 else 6
            self.ball.vx = nx * kick_power + player.vx * 0.5
            self.ball.vy = ny * kick_power + player.vy * 0.5
            player.kick_animation = 0

    def _constrain_player(self, player):
        margin = player.radius
        if player.x - margin < self.field_left:
            player.x = self.field_left + margin
            player.vx = 0
        if player.x + margin > self.field_right:
            player.x = self.field_right - margin
            player.vx = 0
        if player.y - margin < self.field_top:
            player.y = self.field_top + margin
            player.vy = 0
        if player.y + margin > self.field_bottom:
            player.y = self.field_bottom - margin
            player.vy = 0

    def _check_goal(self):
        in_goal_y = (self.goal_y - self.goal_height/2 < self.ball.y < self.goal_y + self.goal_height/2)

        if in_goal_y and self.ball.x < self.field_left:
            self.bloop_score += 1
            self._reset_after_goal()
            return 'bloop_scored'

        if in_goal_y and self.ball.x > self.field_right:
            self.blip_score += 1
            self._reset_after_goal()
            return 'blip_scored'

        return None

    def _reset_after_goal(self):
        self.ball.reset()
        self.blip.x = self.blip.start_x
        self.blip.y = self.blip.start_y
        self.bloop.x = self.bloop.start_x
        self.bloop.y = self.bloop.start_y

    def get_winner(self):
        if self.blip_score > self.bloop_score:
            return 'blip'
        elif self.bloop_score > self.blip_score:
            return 'bloop'
        return 'draw'

print("✅ Game simulation ready")

## Actions

In [ ]:
ACTIONS = [
    {'dx': 0, 'dy': -1, 'kick': False, 'name': 'up'},
    {'dx': 0, 'dy': 1, 'kick': False, 'name': 'down'},
    {'dx': -1, 'dy': 0, 'kick': False, 'name': 'left'},
    {'dx': 1, 'dy': 0, 'kick': False, 'name': 'right'},
    {'dx': -1, 'dy': -1, 'kick': False, 'name': 'up-left'},
    {'dx': 1, 'dy': -1, 'kick': False, 'name': 'up-right'},
    {'dx': -1, 'dy': 1, 'kick': False, 'name': 'down-left'},
    {'dx': 1, 'dy': 1, 'kick': False, 'name': 'down-right'},
    {'dx': 0, 'dy': 0, 'kick': True, 'name': 'kick'},
    {'dx': 0, 'dy': 0, 'kick': False, 'name': 'stay'}
]

def apply_action(player, action):
    if action['dx'] != 0 or action['dy'] != 0:
        player.move(action['dx'], action['dy'])
    if action['kick']:
        player.kick()

print(f"✅ {len(ACTIONS)} actions defined")

## Simple AI Opponent

In [ ]:
class SimpleAI:
    def __init__(self, team):
        self.team = team
        self.last_action = 0

    def get_state(self, player, ball, opponent, field_width, field_height):
        player_x = player.x / field_width
        player_y = player.y / field_height
        ball_x = ball.x / field_width
        ball_y = ball.y / field_height
        ball_vx = np.clip(ball.vx / 15, -1, 1)
        ball_vy = np.clip(ball.vy / 15, -1, 1)
        opp_x = opponent.x / field_width
        opp_y = opponent.y / field_height

        max_dist = np.sqrt(field_width**2 + field_height**2)
        dist_ball = np.sqrt((player.x - ball.x)**2 + (player.y - ball.y)**2) / max_dist
        angle_ball = (np.arctan2(ball.y - player.y, ball.x - player.x) + np.pi) / (2 * np.pi)

        goal_x = field_width if self.team == 'blip' else 0
        goal_y = field_height / 2
        dist_goal = np.sqrt((player.x - goal_x)**2 + (player.y - goal_y)**2) / max_dist
        angle_goal = (np.arctan2(goal_y - player.y, goal_x - player.x) + np.pi) / (2 * np.pi)

        return np.array([player_x, player_y, ball_x, ball_y, ball_vx, ball_vy,
                        opp_x, opp_y, dist_ball, angle_ball, dist_goal, angle_goal])

    def choose_action(self, state):
        player_x, player_y = state[0], state[1]
        ball_x, ball_y = state[2], state[3]
        dist_to_ball = state[8]

        dx_to_ball = ball_x - player_x
        dy_to_ball = ball_y - player_y

        if dist_to_ball < 0.08:
            goal_dir = -1 if self.team == 'bloop' else 1
            if dist_to_ball < 0.04:
                self.last_action = 8
                return ACTIONS[8]
            move_dx = goal_dir
            move_dy = 1 if dy_to_ball > 0.01 else (-1 if dy_to_ball < -0.01 else 0)
            for i in range(8):
                if ACTIONS[i]['dx'] == move_dx and ACTIONS[i]['dy'] == move_dy:
                    self.last_action = i
                    return ACTIONS[i]

        threshold = 0.02
        move_dx = 1 if dx_to_ball > threshold else (-1 if dx_to_ball < -threshold else 0)
        move_dy = 1 if dy_to_ball > threshold else (-1 if dy_to_ball < -threshold else 0)

        for i in range(8):
            if ACTIONS[i]['dx'] == move_dx and ACTIONS[i]['dy'] == move_dy:
                self.last_action = i
                return ACTIONS[i]

        return ACTIONS[3]

    def reset(self):
        pass

print("✅ SimpleAI opponent ready")

## Dueling DQN Agent

In [ ]:
class DuelingDQNAgent:
    def __init__(self, team):
        self.team = team
        self.state_size = 12
        self.action_size = 10

        self.learning_rate = 0.0005
        self.gamma = 0.995
        self.epsilon = 1.0
        self.epsilon_min = 0.02
        self.epsilon_decay = 0.9999

        self.memory = deque(maxlen=50000)
        self.batch_size = 64
        self.min_memory_size = 500

        self.target_update_freq = 500
        self.train_step = 0

        self.model = self._build_model()
        self.target_model = self._build_model()
        self.update_target_network()

        self.last_dist_to_ball = None
        self.last_action = 0

    def _build_model(self):
        inputs = layers.Input(shape=(self.state_size,))

        x = layers.Dense(256, activation='relu', kernel_initializer='he_normal')(inputs)
        x = layers.Dense(256, activation='relu', kernel_initializer='he_normal')(x)
        x = layers.Dense(128, activation='relu', kernel_initializer='he_normal')(x)

        # Value stream
        value = layers.Dense(64, activation='relu', kernel_initializer='he_normal')(x)
        value = layers.Dense(1, kernel_initializer='he_normal', name='value')(value)

        # Advantage stream
        advantage = layers.Dense(64, activation='relu', kernel_initializer='he_normal')(x)
        advantage = layers.Dense(self.action_size, kernel_initializer='he_normal', name='advantage')(advantage)

        # Combine using Keras 3 compatible ops
        mean_adv = layers.Lambda(lambda a: ops.mean(a, axis=1, keepdims=True))(advantage)
        adv_centered = layers.Subtract()([advantage, mean_adv])
        q_values = layers.Add()([value, adv_centered])

        model = keras.Model(inputs=inputs, outputs=q_values)
        model.compile(optimizer=keras.optimizers.Adam(learning_rate=self.learning_rate), loss='mse')
        return model

    def update_target_network(self):
        self.target_model.set_weights(self.model.get_weights())

    def get_state(self, player, ball, opponent, field_width, field_height):
        player_x = player.x / field_width
        player_y = player.y / field_height
        ball_x = ball.x / field_width
        ball_y = ball.y / field_height
        ball_vx = np.clip(ball.vx / 15, -1, 1)
        ball_vy = np.clip(ball.vy / 15, -1, 1)
        opp_x = opponent.x / field_width
        opp_y = opponent.y / field_height

        max_dist = np.sqrt(field_width**2 + field_height**2)
        dist_ball = np.sqrt((player.x - ball.x)**2 + (player.y - ball.y)**2) / max_dist
        angle_ball = (np.arctan2(ball.y - player.y, ball.x - player.x) + np.pi) / (2 * np.pi)

        goal_x = field_width if self.team == 'blip' else 0
        goal_y = field_height / 2
        dist_goal = np.sqrt((player.x - goal_x)**2 + (player.y - goal_y)**2) / max_dist
        angle_goal = (np.arctan2(goal_y - player.y, goal_x - player.x) + np.pi) / (2 * np.pi)

        return np.array([player_x, player_y, ball_x, ball_y, ball_vx, ball_vy,
                        opp_x, opp_y, dist_ball, angle_ball, dist_goal, angle_goal])

    def choose_action(self, state, training=True):
        if training and np.random.random() < self.epsilon:
            action_idx = np.random.randint(self.action_size)
        else:
            q_values = self.model.predict(state.reshape(1, -1), verbose=0)
            action_idx = np.argmax(q_values[0])
        self.last_action = action_idx
        return ACTIONS[action_idx]

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def train(self):
        if len(self.memory) < self.min_memory_size:
            return

        batch = random.sample(self.memory, self.batch_size)
        states = np.array([e[0] for e in batch])
        next_states = np.array([e[3] for e in batch])

        current_qs = self.model.predict(states, verbose=0)
        next_qs_main = self.model.predict(next_states, verbose=0)
        next_qs_target = self.target_model.predict(next_states, verbose=0)

        for i, (state, action, reward, next_state, done) in enumerate(batch):
            if done:
                current_qs[i][action] = reward
            else:
                best_action = np.argmax(next_qs_main[i])
                current_qs[i][action] = reward + self.gamma * next_qs_target[i][best_action]

        self.model.fit(states, current_qs, epochs=1, verbose=0)

        self.train_step += 1
        if self.train_step % self.target_update_freq == 0:
            self.update_target_network()

    def calculate_reward(self, player, ball, opponent, event, field_width):
        reward = 0
        field_height = 420
        dist_to_ball = np.sqrt((player.x - ball.x)**2 + (player.y - ball.y)**2)

        if event == 'scored':
            reward += 500
        elif event == 'conceded':
            reward -= 300

        max_dist = np.sqrt(field_width**2 + field_height**2)
        normalized_dist = dist_to_ball / max_dist
        reward += (1 - normalized_dist) * 5

        if dist_to_ball < 40:
            reward += 10

        if self.last_dist_to_ball is not None:
            dist_delta = self.last_dist_to_ball - dist_to_ball
            reward += dist_delta * 0.5
            if dist_delta > 2:
                reward += 3
        self.last_dist_to_ball = dist_to_ball

        player_speed = np.sqrt(player.vx**2 + player.vy**2)
        if player_speed < 0.5 and dist_to_ball > 50:
            reward -= 8
        if player_speed > 1:
            reward += 1

        attacking_goal_x = field_width if self.team == 'blip' else 0
        ball_moving_toward_goal = (self.team == 'blip' and ball.vx > 2) or (self.team == 'bloop' and ball.vx < -2)
        if ball_moving_toward_goal and dist_to_ball < 80:
            reward += 8

        dist_ball_to_goal = abs(ball.x - attacking_goal_x)
        if dist_ball_to_goal < 100:
            reward += 5

        corner_margin = 80
        in_corner = (player.x < corner_margin or player.x > field_width - corner_margin) and \
                    (player.y < corner_margin or player.y > field_height - corner_margin)
        if in_corner:
            reward -= 5
            if dist_to_ball > 100:
                reward -= 5

        if dist_to_ball > 300:
            reward -= 5
        elif dist_to_ball > 200:
            reward -= 3
        elif dist_to_ball > 150:
            reward -= 1

        opp_dist = np.sqrt((opponent.x - ball.x)**2 + (opponent.y - ball.y)**2)
        if opp_dist < dist_to_ball and dist_to_ball > 60:
            reward -= 2

        reward -= 0.1
        return reward

    def reset(self):
        self.last_dist_to_ball = None
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

    def save_weights(self, filepath):
        weights = self.model.get_weights()
        weight_data = [{'shape': list(w.shape), 'data': w.flatten().tolist()} for w in weights]
        save_data = {'weights': weight_data, 'epsilon': self.epsilon, 'trainStepCount': self.train_step}
        with open(filepath, 'w') as f:
            json.dump(save_data, f)

print("✅ Dueling DQN Agent ready")

## Training Loop

In [ ]:
def train(num_episodes=100000, match_time=30, save_every=5000):
    game = Game(match_time=match_time)
    blip_agent = DuelingDQNAgent('blip')
    bloop_ai = SimpleAI('bloop')

    stats = {'blip_wins': 0, 'bloop_wins': 0, 'draws': 0, 'total_goals': 0}
    start_time = time.time()

    print("=" * 50)
    print("🤖 RL Football Champions - GPU Training")
    print("=" * 50)
    print(f"Episodes: {num_episodes}")
    print(f"Match time: {match_time}s")
    print("Opponent: SimpleAI (ball-chaser)")
    print("=" * 50)

    for episode in range(1, num_episodes + 1):
        game.reset()
        game.reset_scores()
        steps = 0
        done = False

        while not done:
            steps += 1
            blip_state = blip_agent.get_state(game.blip, game.ball, game.bloop, game.field_width, game.field_height)
            bloop_state = bloop_ai.get_state(game.bloop, game.ball, game.blip, game.field_width, game.field_height)

            blip_action = blip_agent.choose_action(blip_state, training=True)
            bloop_action = bloop_ai.choose_action(bloop_state)

            apply_action(game.blip, blip_action)
            apply_action(game.bloop, bloop_action)

            event, done = game.update()

            blip_event = None
            if event == 'blip_scored':
                blip_event = 'scored'
                stats['total_goals'] += 1
            elif event == 'bloop_scored':
                blip_event = 'conceded'
                stats['total_goals'] += 1

            blip_reward = blip_agent.calculate_reward(game.blip, game.ball, game.bloop, blip_event, game.field_width)
            new_blip_state = blip_agent.get_state(game.blip, game.ball, game.bloop, game.field_width, game.field_height)
            blip_agent.remember(blip_state, blip_agent.last_action, blip_reward, new_blip_state, done)

            if steps % 4 == 0:
                blip_agent.train()

        winner = game.get_winner()
        if winner == 'blip':
            stats['blip_wins'] += 1
        elif winner == 'bloop':
            stats['bloop_wins'] += 1
        else:
            stats['draws'] += 1

        blip_agent.reset()
        bloop_ai.reset()

        if episode % 100 == 0 or episode == 1:
            elapsed = time.time() - start_time
            eps_per_sec = episode / elapsed
            remaining = (num_episodes - episode) / eps_per_sec if eps_per_sec > 0 else 0
            eta = f"{remaining/60:.1f}m" if remaining < 3600 else f"{remaining/3600:.1f}h"
            print(f"Episode {episode}/{num_episodes} | ε: {blip_agent.epsilon:.3f} | Blip: {stats['blip_wins']} | Bloop: {stats['bloop_wins']} | Draws: {stats['draws']} | Goals: {stats['total_goals']} | Speed: {eps_per_sec:.1f} ep/s | ETA: {eta}")

        if episode % save_every == 0:
            blip_agent.save_weights(f'blip_weights_{episode}.json')
            print(f"💾 Saved weights at episode {episode}")

    blip_agent.save_weights('blip_weights_final.json')
    print("=" * 50)
    print("✅ Training complete!")
    print(f"Final stats: Blip {stats['blip_wins']} | Bloop {stats['bloop_wins']} | Draws {stats['draws']}")
    print(f"Total goals: {stats['total_goals']}")
    print("=" * 50)
    return blip_agent, stats

print("✅ Training function ready")

## 🚀 Start Training!

In [ ]:
agent, stats = train(num_episodes=100000, match_time=30, save_every=5000)

## Download Weights

In [ ]:
from google.colab import files

# Create browser-compatible format
with open('blip_weights_final.json', 'r') as f:
    blip_data = json.load(f)

browser_data = {
    'version': 2,
    'aiType': 'dqn',
    'episodeCount': 100000,
    'trainedAgainst': 'SimpleAI',
    'stats': stats,
    'blipAgent': blip_data,
    'bloopAgent': blip_data,
    'blip': blip_data,
    'bloop': blip_data
}

with open('rl_football_trained.json', 'w') as f:
    json.dump(browser_data, f)

print("✅ Browser-compatible weights saved!")
files.download('rl_football_trained.json')